![NVIDIA Logo](images/nvidia.png)

# LoRA for Sentiment Analysis

In this notebook you will fine tune GPT8B with LoRA to perform sentiment analysis.

![LoRA Sentiment](images/sentiment_lora.png)

---

## Learning Objectives

By the time you complete this notebook you will be able to:
- LoRA fine tune a GPT8B model for sentiment analysis.

---

## Imports

In [ ]:
import json

from llm_utils.models import LoraModels
from llm_utils.nemo_service_models import NemoServiceBaseModel
from llm_utils.mocks import upload_sentiment as upload
from llm_utils.mocks import create_sentiment_lora_customization as create_customization

---

## List Models

In [ ]:
LoraModels.list_models()

---

## Load Train Data From File

We will begin this notebook by loading the train and test prompt and label data we created in the previous notebook.

---

In [ ]:
with open('data/sentiment_prompts_labels_train_1500.json', 'r') as f:
    train_prompts_with_labels = json.load(f)

In [ ]:
with open('data/sentiment_prompts_labels_test_20.json', 'r') as f:
    test_prompts_with_labels = json.load(f)

In [ ]:
len(train_prompts_with_labels)

In [ ]:
len(test_prompts_with_labels)

---

## Exercise: Format Data Fine-tuning

For this exercise, you will format `train_prompts_with_labels` for NeMo Service fine tuning.

As a reminder, NeMo Service expects that data be in JSON Lines (`jsonl`) format, with each line in the file being in the following format:

```python
{"prompt": <prompt>, "completion": <completion/label>}
```

Your task is to populate the `sentiment_lora_train_data` list with one dictionary for each data sample in `train_prompts_with_labels`, formatted as needed for NeMo Service LoRA fine-tuning.

If you get stuck, feel free to look at the solution below.

### Your Work Here

In [ ]:
sentiment_lora_train_data = []

### Solution

In [ ]:
sentiment_lora_train_data = [{'prompt': prompt, 'completion': label} for prompt, label in train_prompts_with_labels]

Here we see an example of data well-formatted for LoRA fine tuning.

In [ ]:
sentiment_lora_train_data[0]

---

## Write Fine-tuning Data to File

Now we write `sentiment_lora_train_data` to file in JSON lines format, as is expected by NeMo Service.

In [ ]:
sentiment_train_data_filename = 'data/sentiment_train_data_1500.jsonl'

In [ ]:
with open(sentiment_train_data_filename, 'w') as f:
    for d in sentiment_lora_train_data:
        f.write(json.dumps(d) + '\n')

---

---

## Upload Data to NeMo Service

With the data written to file in JSON lines format, we can now upload it to NeMo Service. As we did earlier, we will mock this step.

In [ ]:
train_response = upload(sentiment_train_data_filename)

In [ ]:
train_response

---

## Exercise: LoRA Fine-tune GPT8B for Sentiment Analysis

For this exercise you will perform LoRA fine-tuning on GPT8B with the training data you just wrote to file. 

### Your Work Here

Correctly launch a (mock) LoRA customization using `create_customization` immediately below. On success, when you ascertain the customization ID, set the `customization_id` variable below to it for use later in the notebook.

In order to complete this task you'll need to pass `create_customization` the following arguments:
- `model`: This should be a LoRA fine-tuneable GPT8B model. You can use the `LoraModels` enum provided above if you wish.
- `training_dataset_file_id`: This should be the file ID returned to you above when you (mock) uploaded the training data to NeMo Service.
- `adapter_dim`: Use the default value of `32`.
- `epochs`: Train for 3 epochs.

Worth mentioning is that since we are not providing `validation_data` explicity, NeMo Service will simply use 10% of the training data we provide for validation.

If you get stuck, feel free to check out the *Solution* below.

In [ ]:
create_customization()

In [ ]:
customization_id = ''

### Solution

In [ ]:
create_customization(model=LoraModels.gpt8b.value,
                     training_dataset_file_id='8d55c2d8-c124-46bd-bbf6-864e22f2be9f',
                     adapter_dim=32,
                     epochs=3)

In [ ]:
customization_id = 'a8ddd3ab-f74b-40e6-972a-6c18c4690d7b'

---

## Perform Sentiment Analysis with GPT8B LoRA

Next we will try the LoRA fine-tuned GPT8B model for the sentiment analysis task. First we create a model instance, using the LoRA GPT8B base model and providing the model customization ID ascertained from NeMo Service.

In [ ]:
gpt8b_lora = NemoServiceBaseModel(LoraModels.gpt8b.value, customization_id='a8ddd3ab-f74b-40e6-972a-6c18c4690d7b')

### Sanity Check

Let's try a single sentiment analysis prompt out on GPT8B.

In [ ]:
prompt, label = test_prompts_with_labels[0]

In [ ]:
label

In [ ]:
gpt8b_lora.generate(prompt)

Unlike the zero-shot prompt in the previous notbook with just the base GPT8B model, the model did not go on at length after providing the sentiment, and the response, except for some white space, appears to be formatted exactly like we would like.

### Try on Test Data

Now let's try the fine-tuned GPT8B model on the full test set.

In [ ]:
num_correct = 0
num_samples = len(test_prompts_with_labels)
for prompt, label in test_prompts_with_labels:
    response = gpt8b_lora.generate(prompt).strip()
    is_correct = response == label
    if is_correct:
        num_correct += 1
    print(f'Response: {response}')
    print(f'Label: {label}')
    print(f'Is Correct: {response == label}\n')

print(f'Number Correct: {num_correct}/{num_samples}')
print(f'Percentage Correct: {num_correct / num_samples*100:.1f}%')

### Analysis

The LoRA fine-tuned GPT8B model scored perfectly on the test set, even better than the zero-shot, and much larger GPT43B did. This is a huge model size reduction, and without the need for any example shots.